<a href="https://colab.research.google.com/gist/virattt/4d764c427892ce9fdf4534209edfb1f4/langgraph-financial-agent-polygon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph
!pip install -U langchain langchain_openai langchainhub
!pip install -U polygon-api-client

In [ ]:
import getpass
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools import PolygonLastQuote

# Get the prompt to use (default
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4")

# Create the PolygonLastQuote tool
polygon = PolygonAPIWrapper(polygon_api_key="YOUR_POLYGON_API_KEY")
tools = [PolygonLastQuote(api_wrapper=polygon)]

agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish


# Define the agent
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    observation = tool_to_use.invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    else:
        return "continue"

In [ ]:
from langgraph.graph import END, Graph

workflow = Graph()

workflow.add_node("agent", agent)
workflow.add_node("tools", execute_tools)

# Set the entrypoint as `agent`
workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "exit": END
    }
)
workflow.add_edge('tools', 'agent')



# This compiles it into a LangChain Runnable,
# meaning we can use it as you would any other runnable
chain = workflow.compile()

In [ ]:
result = chain.invoke({"input": "What is the latest stock price for AAPL, MSFT, and AMZN?", "intermediate_steps": []})
output = result['agent_outcome'].return_values["output"]
print(output)

The latest stock prices are:
- Apple (AAPL): $194.14
- Microsoft (MSFT): $404.00
- Amazon (AMZN): $157.5
